### Ejercicios del tema de Clasificación

*Hugo Díaz Díaz* (*hdiazd00@estudiantes.unileon.es*)

*Correo profesional: hugo.didi.contacto@gmail.com*

---

## Parte teórica (opcional)

### 1. ¿Cuál es la profundidad aproximada de un Árbol de Decisión entrenado (sin restricciones) en un conjunto de entrenamiento con 1 millón de muestras?

Cuando un árbol de decisión se entrena sin ninguna limitación, sigue dividiendo mientras encuentre una reducción de impureza. Como en cada división las muestras se van partiendo en subconjuntos cada vez más pequeños, el árbol va profundizando nivel a nivel hasta que ya no quedan mejoras posibles.

Con un conjunto tan grande como uno de un millón de instancias, este proceso suele dar lugar a árboles profundos, con muchas decenas de niveles. Por tanto, la profundidad aproximada estará en ese orden: varias decenas de niveles.

### 2. La impureza Gini de un nodo, ¿es generalmente menor o mayor que la de su padre? ¿Es generalmente menor/mayor, o siempre menor/mayor?

Cada división intenta reducir la impureza respecto al nodo padre, así que lo normal es que los hijos tengan una impureza menor. Sin embargo, no es estrictamente obligatorio que sea siempre menor en ambos hijos: lo que sí garantiza el algoritmo es que la impureza ponderada de los dos hijos será menor que la del padre. Por tanto, generalmente es menor, pero no siempre en todos los nodos individuales.

### 3. Si un Árbol de Decisión está sobreajustando el conjunto de entrenamiento, ¿es una buena idea intentar disminuir `max_depth`?

Sí, totalmente. Reducir la profundidad máxima limita la complejidad del árbol y actúa como regularización, así que disminuir `max_depth` es una de las formas clásicas de combatir el *overfitting*.

### 4. Si un Árbol de Decisión se ajusta demasiado poco al conjunto de entrenamiento, ¿es una buena idea intentar escalar las características de entrada?

No, los árboles no dependen de la escala de los atributos, así que escalar no va a mejorar el ajuste. Si el modelo está infraajustando, lo adecuado sería relajar la regularización (permitir más profundidad, más hojas, etc.). 

### 5. Si se tarda una hora en entrenar un Árbol de Decisión en un conjunto de entrenamiento que contiene 1 millón de instancias, ¿cuánto tiempo se tardará aproximadamente en entrenar otro Árbol de Decisión en un conjunto de entrenamiento que contiene 10 millones de instancias?

El tiempo de entrenamiento de un árbol de decisión crece aproximadamente como $n \cdot \log n$, porque el árbol procesa todas las muestras y además su profundidad aumenta de forma logarítmica con el tamaño del dataset. Al pasar de 1 millón a 10 millones de instancias, el factor dominante es el salto lineal: multiplicas por diez la cantidad de datos. El término logarítmico también sube, pero muy poco (de unos 20 niveles a 23).

En conjunto, el tiempo final será aproximadamente diez veces mayor, así que si antes tardaba 1 hora, con 10 millones tardará alrededor de unas 10 horas.

### 6. Si su conjunto de entrenamiento contiene 100.000 instancias, ¿acelerará el entrenamiento si establece `presort=True`?

No. El preordenamiento solo es útil para datasets pequeños. A partir de unas decenas de miles de instancias, el coste del presort hace que en realidad vaya más lento. Con 100.000 muestras no aporta mejoras.

## Parte práctica (obligatoria)

### 1. Entrena y ajusta un Árbol de Decisión para el conjunto de datos Moons.

In [10]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# For reproducibility
np.random.seed(42) 

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

#### a) Genera un conjunto de datos Moons utilizando make_moons(`n_samples=10000`, `noise=0.4`).

Lo primero es generar el dataset sintético tipo “moons” con bastante ruido para que el problema no sea trivial. 

In [11]:
from sklearn.datasets import make_moons

# Genero el conjunto de datos Moons con 10.000 muestras y ruido 0.4
X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

print(X.shape, y.shape)

(10000, 2) (10000,)


Esto representan 10.000 puntos bidimensionales y sus etiquetas binarias correspondientes.

#### b) Divídelo en un conjunto de entrenamiento y un conjunto de test usando `train_test_split()`.

Ahora divido los datos en train y test. Uso un 80/20 típico y fijo también `random_state` para que la partición sea estable. Aprovecho para estratificar según la clase y mantener proporciones similares en ambos conjuntos.

In [12]:
from sklearn.model_selection import train_test_split

# Divido en entrenamiento y test, 80% / 20%
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(X_train.shape, X_test.shape)

(8000, 2) (2000, 2)


Con esto ya está listo el entorno básico: datos generados y partición hecha.

#### c) Utiliza `GridSearchCV` para encontrar buenos valores de hiperparámetros para un `DecisionTreeClassifier`. Sugerencia: pruebe varios valores para `max_leaf_nodes`.

#### d) Entrénalo con el conjunto de entrenamiento completo utilizando estos hiperparámetros y mide el rendimiento de su modelo en el conjunto de pruebas. Debería obtener entre un 85% y un 87% de exactitud.

### 2. Cultivar un bosque.

#### a) Continuando con el ejercicio anterior, genere 1.000 subconjuntos del conjunto de entrenamiento, cada uno con 100 muestras seleccionadas aleatoriamente. Sugerencia: para ello puedes utilizar la clase `ShuffleSplit` de Scikit-Learn.

#### b) Entrena un Árbol de Decisión en cada subconjunto, utilizando los mejores valores de hiperparámetros encontrados anteriormente. Evalúe estos 1.000 árboles de decisión en el conjunto de test. Dado que fueron entrenados en conjuntos más pequeños, estos Árboles de Decisión probablemente funcionarán peor que el primer Árbol de Decisión, alcanzando sólo un 80% de exactitud.

#### c) Para cada muestra, genera las predicciones de los 1.000 árboles de decisión y conserva sólo la predicción más frecuente (puedes usar la función `mode()` de SciPy). Así obtendrás predicciones mayoritarias sobre el conjunto de prueba.

#### d) Evalúa estas predicciones en el conjunto de test: deberías obtener una exactitud ligeramente superior a la de tu primer modelo (entre un 0,5 y un 1,5% más). Enhorabuena, has entrenado un clasificador Random Forest.